In [ ]:
import sys
sys.path.append('/autofs/homes/005/fd881/repos/MedImaging-ModelDriftMonitoring/')

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
from pathlib import Path
import pandas as pd

#import click
#import plotnine
from pycrumbs import tracked

from src.model_drift.data import mgb_data
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sn
import matplotlib.dates as mdates

In [ ]:
#perf_cols = [
#    col for col in df.columns
#    if  col[0].startswith('performance')
#]

In [ ]:
df = pd.read_csv("/autofs/cluster/qtim/projects/xray_drift/drift_analyses/ER_jackknife/output.csv", header=[0, 1, 2, 3])
# The date column gets read in with a stupid name
date_col = tuple(f'Unnamed: 0_level_{i}' for i in range(4))
performance_col = ('performance', 'micro avg', 'auroc', 'mean')

df[date_col] = pd.to_datetime(df[date_col])
df.head()

In [ ]:
date_col_df = df[date_col]

In [ ]:
fig, ax = plt.subplots()

# make the figure larger
fig.set_size_inches(12, 6)
# Plot the mean line
ax.plot(df[date_col], df[('full_mu', 'emd_jackknife', 'distance', 'mean')], label='Mean MMC')

# Fill the area between the min and max lines
ax.fill_between(df[date_col], df[('full_mu', 'emd_jackknife', 'distance', 'min')], df[('full_mu', 'emd_jackknife', 'distance', 'max')], 
                color='gray', alpha=0.5, label='Min-Max Range')


plt.legend()
plt.xticks(rotation=45)
#start the plot in october 2019
plt.xlim(pd.to_datetime('2019-10-01'), pd.to_datetime('2021-07-01'))

ax.xaxis.set_major_locator(mdates.MonthLocator(interval=3))
ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))

plt.xlabel('Date')
plt.title('EMD with JackKnife over time')
plt.show()

In [ ]:
mmc_cols = [
    col for col in df.columns
    #if not col[0].startswith('performance')
    if not col[0].startswith('mu')
    #and col[1] == 'ks_jackknife'
    and col[2] == 'distance'
    and col[3] == 'mean'
    #drop bitsstored and pixelrepresentation there is something wrong with it
    and col[0] != 'BitsStored'
    and col[0] != 'PixelRepresentation'
]

mmc_cols_min = [
    col for col in df.columns
    #if not col[0].startswith('performance')
    if not col[0].startswith('mu')
    #and col[1] == 'ks_jackknife'
    and col[2] == 'distance'
    and col[3] == 'min'
    #drop bitsstored and pixelrepresentation there is something wrong with it
    and col[0] != 'BitsStored'
    and col[0] != 'PixelRepresentation'
]

mmc_cols_max = [
    col for col in df.columns
    #if not col[0].startswith('performance')
    if not col[0].startswith('mu')
    #and col[1] == 'ks_jackknife'

    and col[2] == 'distance'
    and col[3] == 'max'
    #drop bitsstored and pixelrepresentation there is something wrong with it
    and col[0] != 'BitsStored'
    and col[0] != 'PixelRepresentation'
]

encoder_cols = [
    col for col in df.columns
    if 'mu.' not in col[0] and 'activation' not in col[0]
    #if 'mu.' in col[0]

    and col[2] == 'distance'
    and col[3] == 'mean'
]

#mmc_df = df[encoder_cols + [date_col]+ [performance_col]].copy()
#mmc_df.insert((len(mmc_df.columns)-2), 'drift_principal', drift_principal[:,0])
mmc_df = df[mmc_cols + [date_col]+ [performance_col]].copy()
mmc_df_min = df[mmc_cols_min + [date_col]+ [performance_col]].copy()
mmc_df_max = df[mmc_cols_max + [date_col]+ [performance_col]].copy()

ref_df_weights = mmc_df[mmc_df[date_col] < mgb_data.VAL_DATE_END].copy()
mmc_df.drop(columns=[performance_col], inplace=True)
mmc_df_min.drop(columns=[performance_col], inplace=True)
mmc_df_max.drop(columns=[performance_col], inplace=True)

#drop bits stored there is something wrong with it
#mmc_df.drop(columns=[('BitsStored', 'chi2', 'distance', 'mean')], inplace=True)


ref_df = mmc_df[mmc_df[date_col] < mgb_data.VAL_DATE_END].copy()
ref_df_min = mmc_df_min[mmc_df_min[date_col] < mgb_data.VAL_DATE_END].copy()
ref_df_max = mmc_df_max[mmc_df_max[date_col] < mgb_data.VAL_DATE_END].copy()


In [ ]:
for c in mmc_cols:
    mmc_df[c] = (mmc_df[c] - ref_df[c].mean()) / (ref_df[c].std() + 1e-6)
    # replace mean word in c with min
    c_list = list(c)
    c_list[-1] = 'min'
    c_min = tuple(c_list)
    mmc_df_min[c_min] = (mmc_df_min[c_min] - ref_df[c].mean()) / (ref_df[c].std() + 1e-6)
    # replace mean word in c with max
    c_list = list(c)
    c_list[-1] = 'max'
    c_max = tuple(c_list)
    mmc_df_max[c_max] = (mmc_df_max[c_max] - ref_df[c].mean()) / (ref_df[c].std() + 1e-6)

mmc_df_max.drop(date_col, axis=1, inplace=True, errors='ignore')
mmc_df_min.drop(date_col, axis=1, inplace=True, errors='ignore')
mmc_df.drop(date_col, axis=1, inplace=True, errors='ignore')



In [ ]:

mmc_plot = mmc_df.iloc[:,:].astype(float)
mmc_plot_min = mmc_df_min.iloc[:,:].astype(float)
mmc_plot_max = mmc_df_max.iloc[:,:].astype(float)

mmc_plot = mmc_plot *  weights[:-1]
mmc_plot_min = mmc_plot_min *  weights[:-1]
mmc_plot_max = mmc_plot_max *  weights[:-1]

mmc_plot['mmc'] = mmc_plot.mean(axis=1)
mmc_plot_min['mmc'] = mmc_plot_min.mean(axis=1)
mmc_plot_max['mmc'] = mmc_plot_max.mean(axis=1)


# add date column
mmc_plot[date_col] = date_col_df
mmc_plot_min[date_col] = date_col_df
mmc_plot_max[date_col] = date_col_df

In [ ]:
fig, ax = plt.subplots()

# make the figure larger
fig.set_size_inches(12, 6)
# make a part of below the line green between october and january, only below the line
#ax.axvspan(pd.to_datetime('2019-10-01'), pd.to_datetime('2020-01-01'), color='green', alpha=0.2)
# Plot the mean line
ax.plot(mmc_plot[date_col], mmc_plot['mmc'], label='Mean MMC')

# Fill the area between the min and max lines
ax.fill_between(mmc_plot[date_col], mmc_plot_min['mmc'], mmc_plot_max['mmc'], 
                color='gray', alpha=0.5, label='Min-Max Range')


plt.legend()
plt.xticks(rotation=45)
#start the plot in october 2019
plt.xlim(pd.to_datetime('2019-10-01'), pd.to_datetime('2021-07-01'))

ax.xaxis.set_major_locator(mdates.MonthLocator(interval=3))
ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))

plt.xlabel('Date')
plt.title('weighted MMC over time with JackKnife Resampling and EMD for MGH IMG XR ER MG WHT1')
plt.show()

In [ ]:
# make uniform list of weights of len of mmc_df
weights_uniform = np.ones(len(mmc_df.columns)) / len(mmc_df.columns)*-1

mmc_plot = mmc_df.iloc[:,:].astype(float)
mmc_plot_min = mmc_df_min.iloc[:,:].astype(float)
mmc_plot_max = mmc_df_max.iloc[:,:].astype(float)

mmc_plot = mmc_plot *  weights_uniform
mmc_plot_min = mmc_plot_min *  weights_uniform
mmc_plot_max = mmc_plot_max *  weights_uniform

mmc_plot['mmc'] = mmc_plot.mean(axis=1)
mmc_plot_min['mmc'] = mmc_plot_min.mean(axis=1)
mmc_plot_max['mmc'] = mmc_plot_max.mean(axis=1)


# add date column
mmc_plot[date_col] = date_col_df
mmc_plot_min[date_col] = date_col_df
mmc_plot_max[date_col] = date_col_df

In [ ]:
fig, ax = plt.subplots()

# make the figure larger
fig.set_size_inches(12, 6)
# make a part of below the line green between october and january, only below the line
#ax.axvspan(pd.to_datetime('2019-10-01'), pd.to_datetime('2020-01-01'), color='green', alpha=0.2)
# Plot the mean line
ax.plot(mmc_plot[date_col], mmc_plot['mmc'], label='Mean MMC')

# Fill the area between the min and max lines
ax.fill_between(mmc_plot[date_col], mmc_plot_min['mmc'], mmc_plot_max['mmc'], 
                color='gray', alpha=0.5, label='Min-Max Range')


plt.legend()
plt.xticks(rotation=45)
#start the plot in october 2019
plt.xlim(pd.to_datetime('2019-10-01'), pd.to_datetime('2021-07-01'))

ax.xaxis.set_major_locator(mdates.MonthLocator(interval=3))
ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))

plt.xlabel('Date')
plt.title('unweighted MMC over time with JackKnife Resampling and EMD')
plt.show()

In [ ]:
#col_to_move = ref_df.pop(date_col)
#ref_df.insert(0, date_col, col_to_move)
#ref_df.head()

In [ ]:
correlation_matrix = ref_df_weights.corr()

# To get correlation with the performance column specifically
performance_correlation = correlation_matrix[performance_col]

performance_correlation_df = pd.DataFrame(performance_correlation)
performance_correlation_df.head(5)


In [ ]:
column_name = performance_correlation_df.columns


In [ ]:
#performance_correlation_df['abs_your_column'] = performance_correlation_df[column_name].abs()  # Create a new column with the absolute values
#df_sorted = performance_correlation_df.sort_values('abs_your_column', ascending=False)    # Sort based on the absolute values#
#df_sorted.drop('abs_your_column', axis=1, inplace=True)  # Optional: Remove the auxiliary column if not needed
#df_sorted.to_csv('/autofs/homes/005/fd881/drift_weights.csv')

In [ ]:
performance_correlation_df.head(20)

In [ ]:
# examine values with largest correlation

print("Largest values:")
print(performance_correlation_df.nlargest(10, column_name))

print("\nSmallest values:")
print(performance_correlation_df.nsmallest(10, column_name))

In [ ]:
plot_df = performance_correlation_df.reset_index()
plot_df.columns = ['_'.join(col).strip() if isinstance(col, tuple) else col for col in plot_df.columns]
plot_df.drop(columns=["level_1___", "level_2___", "level_3___"], inplace=True)
plot_df.columns = ["Metric", "Avg_F1_mean"]

In [ ]:
weights_raw= plot_df["Avg_F1_mean"].to_list()
weights = [abs(x) for x in weights_raw]

#replace nan values with 0
weights = [0 if np.isnan(x) else x for x in weights]

# normalize and take negative value -> should that be applied before?
weights = [(-1)*weight/sum(weights) for weight in weights]


In [ ]:
sn.barplot(x=plot_df.columns[0], y='Avg_F1_mean', data=plot_df)
plt.tick_params(axis='x', rotation=45)

In [ ]:
mmc_df_backup = mmc_df.copy()
mmc_df = mmc_df.iloc[:,:].astype(float)

In [ ]:
weighted_df = mmc_df * weights[:-1]
weighted_df

In [ ]:
weighted_df['mmc'] = weighted_df.mean(axis=1)
weighted_df = weighted_df.join(date_col_df)

In [ ]:
smoothing_window = 5

large_smoothing_window = 50
delay = round(large_smoothing_window/2)
weighted_df['mmc_moving'] = weighted_df['mmc'].rolling(window=smoothing_window).mean()
weighted_df['mmc_moving_100'] = weighted_df['mmc'].rolling(window=large_smoothing_window).mean()
weighted_df['filtered_adjusted'] = weighted_df['mmc_moving_100'].shift(-delay)


mmc_df_backup['mmc_moving'] = mmc_df_backup['mmc'].rolling(window=smoothing_window).mean()
mmc_df_backup['mmc_moving_100'] = mmc_df_backup['mmc'].rolling(window=large_smoothing_window).mean()
mmc_df_backup['filtered_adjusted'] = mmc_df_backup['mmc_moving_100'].shift(-delay)

df['auroc_moving'] = df[performance_col].rolling(window=smoothing_window).mean()


sn.lineplot(x=weighted_df[date_col], y=weighted_df['mmc_moving_100'])
plt.xticks(rotation=45)  # You can adjust the rotation angle as needed
plt.xlabel('Date', color='black')
plt.show()

In [ ]:
fig, ax1 = plt.subplots()

sn.lineplot(x=weighted_df[date_col], y=weighted_df['mmc'], ax=ax1, color='b', alpha=0.7)
sn.lineplot(x=weighted_df[date_col], y=weighted_df['filtered_adjusted'], ax=ax1, color='grey', alpha=0.7)


ax1.set_ylabel('MMC', color='b')
ax1.tick_params(axis='x', rotation=45)
ax1.set_xlabel('Date', color='black')

ax2 = ax1.twinx()

sn.lineplot(x=df[date_col], y=df[performance_col], ax=ax2, color='r', alpha=0.7)

ax2.set_ylabel('AUROC', color='r')


plt.xticks(rotation=45)  # You can adjust the rotation angle as needed
plt.show()

In [ ]:
fig, ax1 = plt.subplots()

sn.lineplot(x=mmc_df_backup[date_col], y=mmc_df_backup['mmc_moving'], ax=ax1, color='b', alpha=0.7)
sn.lineplot(x=mmc_df_backup[date_col], y=mmc_df_backup['mmc_moving'], ax=ax1, color='b', alpha=0.7)

ax1.set_ylabel('MMC', color='b')
ax1.tick_params(axis='x', rotation=45)
ax1.set_xlabel('Date', color='black')

ax2 = ax1.twinx()

sn.lineplot(x=df[date_col], y=df['auroc_moving'], ax=ax2, color='r', alpha=0.7)
ax2.set_ylabel('AUROC', color='r')


plt.xticks(rotation=45)  # You can adjust the rotation angle as needed
plt.show()

In [ ]:
fig, ax1 = plt.subplots()

sn.lineplot(x=mmc_df_backup[date_col], y=mmc_df_backup['mmc'], ax=ax1, color='b', alpha=0.7)
ax1.set_ylabel('MMC', color='b')
ax1.tick_params(axis='x', rotation=45)
ax1.set_xlabel('Date', color='black')

ax2 = ax1.twinx()

sn.lineplot(x=weighted_df[date_col], y=weighted_df['mmc'], ax=ax2, color='g', alpha=0.7)
ax2.set_ylabel('weighted MMC', color='g')


plt.xticks(rotation=45)  # You can adjust the rotation angle as needed
plt.show()

In [ ]:

fig, ax1 = plt.subplots()

sn.lineplot(x=df[date_col], y=df[('performance', 'micro avg', 'auroc', 'mean')
], ax=ax1, color='b', alpha=0.7, label="micro AUROC")
ax1.set_ylabel('Score', color='black')
ax1.tick_params(axis='x', rotation=45)
ax1.set_xlabel('Date', color='black')

ax2 = ax1.twinx()

sn.lineplot(x=df[date_col], y=df[('performance', 'macro avg', 'auroc', 'mean')
], ax=ax1, color='r', alpha=0.7, label="macro AUROC")

sn.lineplot(x=df[date_col], y=df[('performance', 'macro avg', 'f1-score', 'mean')
], ax=ax1, color='g', alpha=0.7, label="f1-score")

plt.legend()

plt.xticks(rotation=45)  # You can adjust the rotation angle as needed
plt.show()

In [ ]:
from sklearn.decomposition import PCA


In [ ]:
pca_drift = PCA(n_components=5)
df_pca = mmc_df.dropna()

In [ ]:
df_pca

In [ ]:
drift_principal = pca_drift.fit_transform(df_pca)

In [ ]:
plt.plot(drift_principal[:,0])

In [ ]:
first_princial = pd.DataFrame(drift_principal[:,0])
first_princial.index = df[date_col]
mmc_df.index

In [ ]:
fig, ax1 = plt.subplots()
plt.plot(first_princial)
ax1.set_ylabel('Score', color='black')
ax1.tick_params(axis='x', rotation=45)
ax1.set_xlabel('Date', color='black')